## 가상 화장품 전면에 들어갈 폰트 이미지 생성하기

In [ ]:
# FONTS_DIR  = Path(r"C:\Users\nam\Desktop\tst\ref_fonts_ttf")
# OUT_ROOT   = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\front_bw")

In [4]:
import json
import re
import sys
import traceback
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont, ImageChops
from fontTools.ttLib import TTFont

# =========================
# ✅ 1. 설정 및 경로
# =========================
FONTS_DIR  = Path(r"C:\Users\nam\Desktop\tst\ref_fonts_ttf")
OUT_ROOT   = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\front_bw")

# 렌더링 옵션
RENDER_FONT_SIZE = 160          # 메인 텍스트 크기
SMALL_FONT_RATIO = 0.6          # 용량 표기 폰트 비율
LINE_GAP = 20                   # 줄 간격 (빈 줄이 들어가므로 간격은 좁혀도 됨)
PAD      = 20
PIXEL_CROP = True
SKIP_MISSING_GLYPHS = True
OUTPUT_SIZE = 1024
SLASH_POLICY = "split"

# ⭐ [설정] 문단 사이에 빈 줄 추가 여부
ADD_EMPTY_LINE = True

# 용량/단위 감지 정규식
VOLUME_PATTERN = re.compile(r"^(.*?)\s*(\d+(?:\.\d+)?\s*(?:ml|mL|g|kg|oz|pads|sheets|ea))\s*$", re.IGNORECASE)

# =========================
# ✅ 2. 데이터 (200개 레코드)
# =========================
RECORDS_JSONL = """
{"lines": ["blurrish calm", "바디 로션 355ml"]}
{"lines": ["blurrish calm", "핸드크림 30ml"]}
{"lines": ["auralis dew", "바디워시 300ml"]}
{"lines": ["auralis dew", "바디 로션 355ml"]}
{"lines": ["daylight pure", "토너 200ml"]}
{"lines": ["daylight pure", "세럼 50ml"]}
{"lines": ["velveton soft", "클렌징 오일 200ml"]}
{"lines": ["velveton soft", "클렌징 폼 120ml"]}
{"lines": ["novafirm zero", "클렌징 워터 300ml"]}
{"lines": ["novafirm zero", "클렌징 젤 200ml"]}
{"lines": ["moonleaf lift", "선크림 60ml"]}
{"lines": ["moonleaf lift", "토너 180ml"]}
{"lines": ["amberra hydra", "선 에센스 50ml"]}
{"lines": ["amberra hydra", "로션 250ml"]}
{"lines": ["prismia cica", "페이스 크림 70g"]}
{"lines": ["prismia cica", "아이 크림 20g"]}
{"lines": ["petaluna bright", "미스트 80ml"]}
{"lines": ["petaluna bright", "토너 170ml"]}
{"lines": ["silkroot fresh", "앰플 30ml"]}
{"lines": ["silkroot fresh", "수딩젤 150ml"]}
{"lines": ["mellowlab matte", "샴푸 300ml"]}
{"lines": ["mellowlab matte", "트리트먼트 400ml"]}
{"lines": ["lunavia glow", "클레이 마스크 100g"]}
{"lines": ["lunavia glow", "멀티밤 15g"]}
{"lines": ["seaseed satin", "립밤 4g"]}
{"lines": ["seaseed satin", "립틴트 3.5g"]}
{"lines": ["skylin herb", "슬리핑 마스크 100g"]}
{"lines": ["skylin herb", "페이스 크림 50g"]}
{"lines": ["berrynote airy", "약산성 폼 150ml"]}
{"lines": ["berrynote airy", "토너 250ml"]}
{"lines": ["hushskin daily", "바디 로션 400ml"]}
{"lines": ["hushskin daily", "핸드크림 75ml"]}
{"lines": ["cloudrin night", "바디워시 355ml"]}
{"lines": ["cloudrin night", "바디 로션 250ml"]}
{"lines": ["oasisle clean", "토너 180ml"]}
{"lines": ["oasisle clean", "세럼 30ml"]}
{"lines": ["glowmint tint", "클렌징 오일 150ml"]}
{"lines": ["glowmint tint", "클렌징 폼 100ml"]}
{"lines": ["serenique soothe", "클렌징 워터 500ml"]}
{"lines": ["serenique soothe", "클렌징 젤 150ml"]}
{"lines": ["purekind vita", "선크림 50ml"]}
{"lines": ["purekind vita", "토너 200ml"]}
{"lines": ["novaferm aqua", "선 에센스 50ml"]}
{"lines": ["novaferm aqua", "로션 200ml"]}
{"lines": ["warmhug velvet", "페이스 크림 50g"]}
{"lines": ["warmhug velvet", "아이 크림 15g"]}
{"lines": ["aromad mild", "미스트 50ml"]}
{"lines": ["aromad mild", "토너 150ml"]}
{"lines": ["cleanwave smooth", "앰플 30ml"]}
{"lines": ["cleanwave smooth", "수딩젤 200ml"]}
{"lines": ["radiantia bloom", "샴푸 500ml"]}
{"lines": ["radiantia bloom", "트리트먼트 300ml"]}
{"lines": ["serenbay cloud", "클레이 마스크 50g"]}
{"lines": ["serenbay cloud", "멀티밤 20g"]}
{"lines": ["ivorylane warm", "립밤 5g"]}
{"lines": ["ivorylane warm", "립틴트 4g"]}
{"lines": ["mintbloom zen", "슬리핑 마스크 100g"]}
{"lines": ["mintbloom zen", "페이스 크림 70g"]}
{"lines": ["coralroom calm", "약산성 폼 120ml"]}
{"lines": ["coralroom calm", "토너 170ml"]}
{"lines": ["pearlhush dew", "바디 로션 250ml"]}
{"lines": ["pearlhush dew", "핸드크림 100ml"]}
{"lines": ["teatree pure", "바디워시 500ml"]}
{"lines": ["teatree pure", "바디 로션 473ml"]}
{"lines": ["sandmuse soft", "토너 150ml"]}
{"lines": ["sandmuse soft", "세럼 15ml"]}
{"lines": ["lumenry zero", "클렌징 오일 170ml"]}
{"lines": ["lumenry zero", "클렌징 폼 150ml"]}
{"lines": ["dewfound lift", "클렌징 워터 200ml"]}
{"lines": ["dewfound lift", "클렌징 젤 180ml"]}
{"lines": ["floraquill hydra", "선크림 40ml"]}
{"lines": ["floraquill hydra", "토너 250ml"]}
{"lines": ["stoneleaf cica", "선 에센스 50ml"]}
{"lines": ["stoneleaf cica", "로션 150ml"]}
{"lines": ["citruskin bright", "페이스 크림 30g"]}
{"lines": ["citruskin bright", "아이 크림 20g"]}
{"lines": ["cloudharbor fresh", "미스트 100ml"]}
{"lines": ["cloudharbor fresh", "토너 200ml"]}
{"lines": ["freshmuse matte", "앰플 30ml"]}
{"lines": ["freshmuse matte", "수딩젤 100ml"]}
{"lines": ["barelythere glow", "샴푸 400ml"]}
{"lines": ["barelythere glow", "트리트먼트 200ml"]}
{"lines": ["softcove satin", "클레이 마스크 100g"]}
{"lines": ["softcove satin", "멀티밤 15g"]}
{"lines": ["neobare herb", "립밤 4g"]}
{"lines": ["neobare herb", "립틴트 3.5g"]}
{"lines": ["bloomix airy", "슬리핑 마스크 100g"]}
{"lines": ["bloomix airy", "페이스 크림 70g"]}
{"lines": ["ambrleaf daily", "약산성 폼 150ml"]}
{"lines": ["ambrleaf daily", "토너 150ml"]}
{"lines": ["oceandawn night", "바디 로션 150ml"]}
{"lines": ["oceandawn night", "핸드크림 50ml"]}
{"lines": ["nudemist clean", "바디워시 300ml"]}
{"lines": ["nudemist clean", "바디 로션 500ml"]}
{"lines": ["pinkharbor tint", "토너 230ml"]}
{"lines": ["pinkharbor tint", "세럼 30ml"]}
{"lines": ["quietlily soothe", "클렌징 오일 200ml"]}
{"lines": ["quietlily soothe", "클렌징 폼 120ml"]}
{"lines": ["lucidseed vita", "클렌징 워터 400ml"]}
{"lines": ["lucidseed vita", "클렌징 젤 150ml"]}
{"lines": ["miraflow aqua", "선크림 60ml"]}
{"lines": ["miraflow aqua", "토너 180ml"]}
{"lines": ["solun velvet", "선 에센스 50ml"]}
{"lines": ["solun velvet", "로션 300ml"]}
{"lines": ["aurevia mild", "페이스 크림 70g"]}
{"lines": ["aurevia mild", "아이 크림 15g"]}
{"lines": ["glaciera smooth", "미스트 50ml"]}
{"lines": ["glaciera smooth", "토너 170ml"]}
{"lines": ["fawnroot bloom", "앰플 30ml"]}
{"lines": ["fawnroot bloom", "수딩젤 150ml"]}
{"lines": ["rosenest cloud", "샴푸 500ml"]}
{"lines": ["rosenest cloud", "트리트먼트 400ml"]}
{"lines": ["opaline warm", "클레이 마스크 50g"]}
{"lines": ["opaline warm", "멀티밤 20g"]}
{"lines": ["candlea zen", "립밤 5g"]}
{"lines": ["candlea zen", "립틴트 4g"]}
{"lines": ["nortex calm", "슬리핑 마스크 100g"]}
{"lines": ["nortex calm", "페이스 크림 50g"]}
{"lines": ["silvaris dew", "약산성 폼 120ml"]}
{"lines": ["silvaris dew", "토너 200ml"]}
{"lines": ["cedarvein pure", "바디 로션 473ml"]}
{"lines": ["cedarvein pure", "핸드크림 30ml"]}
{"lines": ["mistralyn soft", "바디워시 400ml"]}
{"lines": ["mistralyn soft", "바디 로션 200ml"]}
{"lines": ["duskelle zero", "토너 150ml"]}
{"lines": ["duskelle zero", "세럼 40ml"]}
{"lines": ["purenova lift", "클렌징 오일 170ml"]}
{"lines": ["purenova lift", "클렌징 폼 100ml"]}
{"lines": ["bluecove hydra", "클렌징 워터 200ml"]}
{"lines": ["bluecove hydra", "클렌징 젤 180ml"]}
{"lines": ["honeyra cica", "선크림 50ml"]}
{"lines": ["honeyra cica", "토너 250ml"]}
{"lines": ["laventh bright", "선 에센스 50ml"]}
{"lines": ["laventh bright", "로션 150ml"]}
{"lines": ["springleaf fresh", "페이스 크림 50g"]}
{"lines": ["springleaf fresh", "아이 크림 15g"]}
{"lines": ["cocomist matte", "미스트 80ml"]}
{"lines": ["cocomist matte", "토너 150ml"]}
{"lines": ["sagehaven glow", "앰플 30ml"]}
{"lines": ["sagehaven glow", "수딩젤 200ml"]}
{"lines": ["breezara satin", "샴푸 300ml"]}
{"lines": ["breezara satin", "트리트먼트 300ml"]}
{"lines": ["ivorydew herb", "클레이 마스크 50g"]}
{"lines": ["ivorydew herb", "멀티밤 20g"]}
{"lines": ["monarchia airy", "립밤 4g"]}
{"lines": ["monarchia airy", "립틴트 3.5g"]}
{"lines": ["pebblebay daily", "슬리핑 마스크 100g"]}
{"lines": ["pebblebay daily", "페이스 크림 70g"]}
{"lines": ["tenderoot night", "약산성 폼 150ml"]}
{"lines": ["tenderoot night", "토너 150ml"]}
{"lines": ["sunwisp clean", "바디 로션 200ml"]}
{"lines": ["sunwisp clean", "핸드크림 50ml"]}
{"lines": ["rainora tint", "바디워시 500ml"]}
{"lines": ["rainora tint", "바디 로션 355ml"]}
{"lines": ["citravale soothe", "토너 170ml"]}
{"lines": ["citravale soothe", "세럼 15ml"]}
{"lines": ["veloria vita", "클렌징 오일 150ml"]}
{"lines": ["veloria vita", "클렌징 폼 150ml"]}
{"lines": ["coralyn aqua", "클렌징 워터 400ml"]}
{"lines": ["coralyn aqua", "클렌징 젤 150ml"]}
{"lines": ["mossmint velvet", "선크림 60ml"]}
{"lines": ["mossmint velvet", "토너 180ml"]}
{"lines": ["lilacove mild", "선 에센스 50ml"]}
{"lines": ["lilacove mild", "로션 200ml"]}
{"lines": ["plumaria smooth", "페이스 크림 30g"]}
{"lines": ["plumaria smooth", "아이 크림 20g"]}
{"lines": ["starling bloom", "미스트 50ml"]}
{"lines": ["starling bloom", "토너 200ml"]}
{"lines": ["mistoria cloud", "앰플 30ml"]}
{"lines": ["mistoria cloud", "수딩젤 150ml"]}
{"lines": ["seaquill warm", "샴푸 400ml"]}
{"lines": ["seaquill warm", "트리트먼트 200ml"]}
{"lines": ["dawnmuse zen", "클레이 마스크 100g"]}
{"lines": ["dawnmuse zen", "멀티밤 15g"]}
{"lines": ["aurorite calm", "립밤 5g"]}
{"lines": ["aurorite calm", "립틴트 4g"]}
{"lines": ["glowridge dew", "슬리핑 마스크 100g"]}
{"lines": ["glowridge dew", "페이스 크림 50g"]}
{"lines": ["silkbay pure", "약산성 폼 120ml"]}
{"lines": ["silkbay pure", "토너 150ml"]}
{"lines": ["cloudmori soft", "바디 로션 300ml"]}
{"lines": ["cloudmori soft", "핸드크림 100ml"]}
{"lines": ["petalfog zero", "바디워시 355ml"]}
{"lines": ["petalfog zero", "바디 로션 473ml"]}
{"lines": ["nectarune lift", "토너 230ml"]}
{"lines": ["nectarune lift", "세럼 40ml"]}
{"lines": ["halcyon hydra", "클렌징 오일 200ml"]}
{"lines": ["halcyon hydra", "클렌징 폼 100ml"]}
{"lines": ["mellowine cica", "클렌징 워터 300ml"]}
{"lines": ["mellowine cica", "클렌징 젤 180ml"]}
{"lines": ["satinelle lift", "선크림 50ml"]}
{"lines": ["satinelle lift", "토너 170ml"]}
{"lines": ["moodleaf hydra", "선 에센스 50ml"]}
{"lines": ["moodleaf hydra", "로션 150ml"]}
{"lines": ["clearhaven cica", "페이스 크림 50g"]}
{"lines": ["clearhaven cica", "아이 크림 20g"]}
{"lines": ["sproutia bright", "미스트 100ml"]}
{"lines": ["sproutia bright", "토너 180ml"]}
{"lines": ["luminote fresh", "앰플 30ml"]}
{"lines": ["luminote fresh", "수딩젤 200ml"]}
""".strip()

# 데이터 로드
raw_records = [json.loads(line) for line in RECORDS_JSONL.splitlines() if line.strip()]

# ID 재할당
records = []
for i, rec in enumerate(raw_records):
    rec['id'] = f"{i+1:05d}"
    records.append(rec)

# Pillow 리샘플링 호환성
try: RESAMPLE_LANCZOS = Image.Resampling.LANCZOS
except AttributeError: RESAMPLE_LANCZOS = Image.LANCZOS

# =========================
# Utils
# =========================
def get_clean_font_name(font_path: Path) -> str:
    stem = font_path.stem
    clean_name = re.sub(r"[^a-zA-Z0-9가-힣]", "", stem)
    return clean_name

def normalize_lines(lines):
    """
    1. 슬래시(/) 분리
    2. 용량 표기 분리
    3. [NEW] 문단 사이 빈 줄(' ') 삽입
    """
    temp_lines = []

    # 1차: 기존 정책 분리
    for line in lines:
        s = str(line).strip()
        if not s: continue
        if SLASH_POLICY == "split" and "/" in s:
            parts = [p.strip() for p in re.split(r"\s*/\s*", s) if p.strip()]
            temp_lines.extend(parts)
        elif SLASH_POLICY == "remove":
            temp_lines.append(re.sub(r"\s*/\s*", " ", s).strip())
        else:
            temp_lines.append(s)

    # 2차: 용량 패턴 분리
    parsed_lines = []
    for line in temp_lines:
        match = VOLUME_PATTERN.match(line)
        if match:
            main_part = match.group(1).strip()
            vol_part = match.group(2).strip()
            if main_part: parsed_lines.append(main_part)
            parsed_lines.append(vol_part)
        else:
            parsed_lines.append(line)

    # 3차: 빈 줄 삽입 (Double Spacing)
    if ADD_EMPTY_LINE and len(parsed_lines) > 0:
        spaced_lines = []
        for i, item in enumerate(parsed_lines):
            spaced_lines.append(item)
            # 마지막 아이템이 아니면 공백(" ") 추가 -> 이게 빈 줄 역할
            if i < len(parsed_lines) - 1:
                spaced_lines.append(" ")
        return spaced_lines

    return parsed_lines

def best_cmap(font_path: Path) -> dict:
    try:
        tt = TTFont(str(font_path), lazy=True)
        cmap = tt["cmap"].getBestCmap() or {}
        try: tt.close()
        except: pass
        return cmap
    except: return {}

def missing_chars(cmap: dict, text: str):
    miss = []
    for ch in set(text):
        if ch.isspace(): continue
        if ord(ch) not in cmap: miss.append(ch)
    return sorted(miss)

def tight_crop_white(img: Image.Image) -> Image.Image:
    if not PIXEL_CROP: return img
    bg = Image.new("RGB", img.size, (255, 255, 255))
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()
    return img.crop(bbox) if bbox else img

def fit_into_square(img: Image.Image, size: int) -> tuple[Image.Image, float, tuple[int,int]]:
    w, h = img.size
    scale = min(size / w, size / h, 1.0)
    if scale < 1.0:
        nw = max(1, int(round(w * scale)))
        nh = max(1, int(round(h * scale)))
        img = img.resize((nw, nh), resample=RESAMPLE_LANCZOS)
        w, h = img.size
    canvas = Image.new("RGB", (size, size), (255, 255, 255))
    ox = (size - w) // 2
    oy = (size - h) // 2
    canvas.paste(img, (ox, oy))
    return canvas, float(scale), (int(ox), int(oy))

def get_text_dimensions(text_string, font):
    dummy = Image.new("RGB", (1, 1))
    draw = ImageDraw.Draw(dummy)
    if hasattr(draw, "textbbox"):
        l, t, r, b = draw.textbbox((0, 0), text_string, font=font)
        return r - l, b - t
    else:
        return draw.textsize(text_string, font)

# =========================
# 렌더링 함수
# =========================
def render_multi_size_block(lines, font_path: Path) -> Image.Image:
    font_main = ImageFont.truetype(str(font_path), RENDER_FONT_SIZE)
    font_small = ImageFont.truetype(str(font_path), int(RENDER_FONT_SIZE * SMALL_FONT_RATIO))

    parsed_lines = []
    total_h = 0
    max_w = 0

    for line in lines:
        # 공백 라인(" ")은 정규식 매치 X -> font_main 사용 -> 빈 줄 높이 확보
        is_vol = bool(VOLUME_PATTERN.match(line))
        font = font_small if is_vol else font_main

        w, h = get_text_dimensions(line, font)

        # 만약 " " 공백 줄이라면 높이는 폰트 크기만큼 보장, 너비는 0에 가까움
        if not line.strip():
            h = RENDER_FONT_SIZE # 강제 높이 지정

        parsed_lines.append({"text": line, "font": font, "w": w, "h": h})
        max_w = max(max_w, w)
        total_h += h

    # 줄 간격 추가
    total_h += (len(lines) - 1) * LINE_GAP if len(lines) > 0 else 0

    img_w = max_w + 2 * PAD
    img_h = total_h + 2 * PAD
    img = Image.new("RGB", (img_w, img_h), (255, 255, 255))
    draw = ImageDraw.Draw(img)

    current_y = PAD
    for item in parsed_lines:
        text = item["text"]
        font = item["font"]
        line_w = item["w"]
        line_h = item["h"]

        x = (img_w - line_w) // 2

        # 공백이 아닌 경우만 실제 그리기
        if text.strip():
            draw.text((x, current_y), text, font=font, fill=(0, 0, 0))

        current_y += line_h + LINE_GAP

    return img

# =========================
# 메인 프로세스
# =========================
def process_single_font_safe(font_path, record_list):
    clean_font_name = get_clean_font_name(font_path) or "UnknownFont"
    out_dir = OUT_ROOT / clean_font_name

    ann_path  = out_dir / "annotations.jsonl"
    if ann_path.exists():
        print(f"⏩ Skipping {clean_font_name}", flush=True)
        return

    cmap = best_cmap(font_path)
    if not cmap:
        print(f"⚠️ Bad font: {clean_font_name}", flush=True)
        return

    out_dir.mkdir(parents=True, exist_ok=True)
    miss_path = out_dir / "missing_glyphs.jsonl"
    saved, skipped = 0, 0

    with ann_path.open("w", encoding="utf-8") as ann_f, miss_path.open("w", encoding="utf-8") as miss_f:
        for rec in record_list:
            rid = rec.get("id")
            raw_lines = rec.get("lines", [])
            lines = normalize_lines(raw_lines)
            text = "\n".join(lines)

            miss = missing_chars(cmap, text)
            if miss:
                miss_f.write(json.dumps({"id": rid, "font": str(font_path), "missing_chars": miss}, ensure_ascii=False) + "\n")
                if SKIP_MISSING_GLYPHS:
                    skipped += 1
                    continue

            try:
                img = render_multi_size_block(lines, font_path)
            except Exception:
                continue

            img = tight_crop_white(img)
            inner_w, inner_h = img.size
            final_img, scale, (ox, oy) = fit_into_square(img, OUTPUT_SIZE)

            filename = f"{clean_font_name}_{rid}.png"
            final_img.save(out_dir / filename, "PNG")

            meta = {
                "id": rid,
                "file_name": filename,
                "image_path": f"{clean_font_name}/{filename}",
                "font_name": clean_font_name,
                "output_size": OUTPUT_SIZE,
                "render_font_size": RENDER_FONT_SIZE,
                "small_font_ratio": SMALL_FONT_RATIO,
                "fit_scale": float(scale),
                "paste_offset": [ox, oy],
                "lines": lines
            }
            ann_f.write(json.dumps(meta, ensure_ascii=False) + "\n")
            saved += 1

    print(f"[{clean_font_name}] Saved: {saved}, Skipped: {skipped}", flush=True)

if __name__ == "__main__":
    OUT_ROOT.mkdir(parents=True, exist_ok=True)
    ttf_paths = sorted([p for p in FONTS_DIR.rglob("*.ttf") if p.is_file()])

    print(f"Records: {len(records)}, Fonts: {len(ttf_paths)}")
    print("--------------------------------------------------")
    for i, font_path in enumerate(ttf_paths):
        print(f"👉 [{i+1}/{len(ttf_paths)}] {font_path.name} ... ", end="", flush=True)
        try: process_single_font_safe(font_path, records)
        except KeyboardInterrupt: break
        except Exception as e: print(f"\n❌ Error: {e}", flush=True)

    print("\n✅ All Done.")

Records: 200, Fonts: 150
--------------------------------------------------
👉 [1/150] AritaBuriKR-Medium.ttf ... [AritaBuriKRMedium] Saved: 200, Skipped: 0
👉 [2/150] AstaSans-Regular.ttf ... [AstaSansRegular] Saved: 200, Skipped: 0
👉 [3/150] Binggrae.ttf ... [Binggrae] Saved: 200, Skipped: 0
👉 [4/150] BinggraeII.ttf ... [BinggraeII] Saved: 200, Skipped: 0
👉 [5/150] BinggraeMelona.ttf ... [BinggraeMelona] Saved: 200, Skipped: 0
👉 [6/150] BinggraeSamanco.ttf ... [BinggraeSamanco] Saved: 200, Skipped: 0
👉 [7/150] BinggraeTaom.ttf ... [BinggraeTaom] Saved: 200, Skipped: 0
👉 [8/150] BMHANNA_11yrs_ttf.ttf ... [BMHANNA11yrsttf] Saved: 200, Skipped: 0
👉 [9/150] BookkGothic_Light.ttf ... [BookkGothicLight] Saved: 200, Skipped: 0
👉 [10/150] BookkMyungjo_Light.ttf ... [BookkMyungjoLight] Saved: 200, Skipped: 0
👉 [11/150] Cafe24Simplehae-v2.0.ttf ... [Cafe24Simplehaev20] Saved: 200, Skipped: 0
👉 [12/150] Cafe24SsurroundAir-v1.1.ttf ... [Cafe24SsurroundAirv11] Saved: 200, Skipped: 0
👉 [13/150] Cafe

103736 extra bytes in post.stringData array


[SpoqaHanSansRegular] Saved: 200, Skipped: 0
👉 [107/150] SpoqaHanSansNeo-Regular.ttf ... [SpoqaHanSansNeoRegular] Saved: 200, Skipped: 0
👉 [108/150] STUNNING.ttf ... [STUNNING] Saved: 200, Skipped: 0
👉 [109/150] SUIT-Variable.ttf ... [SUITVariable] Saved: 200, Skipped: 0
👉 [110/150] The Jamsil 3 Regular.ttf ... [TheJamsil3Regular] Saved: 200, Skipped: 0
👉 [111/150] THEFACESHOP+INKLIPQUID(윈도우용).ttf ... [THEFACESHOPINKLIPQUID윈도우용] Saved: 200, Skipped: 0
👉 [112/150] Tlab신영복체.ttf ... [Tlab신영복체] Saved: 200, Skipped: 0
👉 [113/150] UhBee Se_hyun.ttf ... [UhBeeSehyun] Saved: 200, Skipped: 0
👉 [114/150] WantedSans-Regular.ttf ... [WantedSansRegular] Saved: 200, Skipped: 0
👉 [115/150] WantedSansVariable.ttf ... [WantedSansVariable] Saved: 200, Skipped: 0
👉 [116/150] YES24GothicR.ttf ... [YES24GothicR] Saved: 200, Skipped: 0
👉 [117/150] YES24MyoungjoR.ttf ... [YES24MyoungjoR] Saved: 200, Skipped: 0
👉 [118/150] YoonChildfundkoreaMinGuk.ttf ... [YoonChildfundkoreaMinGuk] Saved: 200, Skipped: 0
👉 [1

12292 extra bytes in post.stringData array


[중나좋체Medium] Saved: 200, Skipped: 0
👉 [150/150] 클립아트코리아_Regular.ttf ... [클립아트코리아Regular] Saved: 200, Skipped: 0

✅ All Done.


## 색상을 다양하게 생성

In [8]:
import json
import re
import sys
import random  # ✅ 추가됨
import traceback
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont, ImageChops
from fontTools.ttLib import TTFont

# =========================
# ✅ 1. 설정 및 경로
# =========================
FONTS_DIR  = Path(r"C:\Users\nam\Desktop\tst\ref_fonts_ttf")
OUT_ROOT   = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\front_color_aug")

# 렌더링 옵션
RENDER_FONT_SIZE = 160          # 메인 텍스트 크기
SMALL_FONT_RATIO = 0.6          # 용량 표기 폰트 비율
LINE_GAP = 20                   # 줄 간격
PAD      = 20
PIXEL_CROP = True
SKIP_MISSING_GLYPHS = True
OUTPUT_SIZE = 1024
SLASH_POLICY = "split"

# ⭐ [설정] 문단 사이에 빈 줄 추가 여부
ADD_EMPTY_LINE = True

# 용량/단위 감지 정규식
VOLUME_PATTERN = re.compile(r"^(.*?)\s*(\d+(?:\.\d+)?\s*(?:ml|mL|g|kg|oz|pads|sheets|ea))\s*$", re.IGNORECASE)

# =========================
# ✅ 2. 데이터 (200개 레코드)
# =========================
RECORDS_JSONL = """
{"lines": ["blurrish calm", "바디 로션 355ml"]}
{"lines": ["blurrish calm", "핸드크림 30ml"]}
{"lines": ["auralis dew", "바디워시 300ml"]}
{"lines": ["auralis dew", "바디 로션 355ml"]}
{"lines": ["daylight pure", "토너 200ml"]}
{"lines": ["daylight pure", "세럼 50ml"]}
{"lines": ["velveton soft", "클렌징 오일 200ml"]}
{"lines": ["velveton soft", "클렌징 폼 120ml"]}
{"lines": ["novafirm zero", "클렌징 워터 300ml"]}
{"lines": ["novafirm zero", "클렌징 젤 200ml"]}
{"lines": ["moonleaf lift", "선크림 60ml"]}
{"lines": ["moonleaf lift", "토너 180ml"]}
{"lines": ["amberra hydra", "선 에센스 50ml"]}
{"lines": ["amberra hydra", "로션 250ml"]}
{"lines": ["prismia cica", "페이스 크림 70g"]}
{"lines": ["prismia cica", "아이 크림 20g"]}
{"lines": ["petaluna bright", "미스트 80ml"]}
{"lines": ["petaluna bright", "토너 170ml"]}
{"lines": ["silkroot fresh", "앰플 30ml"]}
{"lines": ["silkroot fresh", "수딩젤 150ml"]}
{"lines": ["mellowlab matte", "샴푸 300ml"]}
{"lines": ["mellowlab matte", "트리트먼트 400ml"]}
{"lines": ["lunavia glow", "클레이 마스크 100g"]}
{"lines": ["lunavia glow", "멀티밤 15g"]}
{"lines": ["seaseed satin", "립밤 4g"]}
{"lines": ["seaseed satin", "립틴트 3.5g"]}
{"lines": ["skylin herb", "슬리핑 마스크 100g"]}
{"lines": ["skylin herb", "페이스 크림 50g"]}
{"lines": ["berrynote airy", "약산성 폼 150ml"]}
{"lines": ["berrynote airy", "토너 250ml"]}
{"lines": ["hushskin daily", "바디 로션 400ml"]}
{"lines": ["hushskin daily", "핸드크림 75ml"]}
{"lines": ["cloudrin night", "바디워시 355ml"]}
{"lines": ["cloudrin night", "바디 로션 250ml"]}
{"lines": ["oasisle clean", "토너 180ml"]}
{"lines": ["oasisle clean", "세럼 30ml"]}
{"lines": ["glowmint tint", "클렌징 오일 150ml"]}
{"lines": ["glowmint tint", "클렌징 폼 100ml"]}
{"lines": ["serenique soothe", "클렌징 워터 500ml"]}
{"lines": ["serenique soothe", "클렌징 젤 150ml"]}
{"lines": ["purekind vita", "선크림 50ml"]}
{"lines": ["purekind vita", "토너 200ml"]}
{"lines": ["novaferm aqua", "선 에센스 50ml"]}
{"lines": ["novaferm aqua", "로션 200ml"]}
{"lines": ["warmhug velvet", "페이스 크림 50g"]}
{"lines": ["warmhug velvet", "아이 크림 15g"]}
{"lines": ["aromad mild", "미스트 50ml"]}
{"lines": ["aromad mild", "토너 150ml"]}
{"lines": ["cleanwave smooth", "앰플 30ml"]}
{"lines": ["cleanwave smooth", "수딩젤 200ml"]}
{"lines": ["radiantia bloom", "샴푸 500ml"]}
{"lines": ["radiantia bloom", "트리트먼트 300ml"]}
{"lines": ["serenbay cloud", "클레이 마스크 50g"]}
{"lines": ["serenbay cloud", "멀티밤 20g"]}
{"lines": ["ivorylane warm", "립밤 5g"]}
{"lines": ["ivorylane warm", "립틴트 4g"]}
{"lines": ["mintbloom zen", "슬리핑 마스크 100g"]}
{"lines": ["mintbloom zen", "페이스 크림 70g"]}
{"lines": ["coralroom calm", "약산성 폼 120ml"]}
{"lines": ["coralroom calm", "토너 170ml"]}
{"lines": ["pearlhush dew", "바디 로션 250ml"]}
{"lines": ["pearlhush dew", "핸드크림 100ml"]}
{"lines": ["teatree pure", "바디워시 500ml"]}
{"lines": ["teatree pure", "바디 로션 473ml"]}
{"lines": ["sandmuse soft", "토너 150ml"]}
{"lines": ["sandmuse soft", "세럼 15ml"]}
{"lines": ["lumenry zero", "클렌징 오일 170ml"]}
{"lines": ["lumenry zero", "클렌징 폼 150ml"]}
{"lines": ["dewfound lift", "클렌징 워터 200ml"]}
{"lines": ["dewfound lift", "클렌징 젤 180ml"]}
{"lines": ["floraquill hydra", "선크림 40ml"]}
{"lines": ["floraquill hydra", "토너 250ml"]}
{"lines": ["stoneleaf cica", "선 에센스 50ml"]}
{"lines": ["stoneleaf cica", "로션 150ml"]}
{"lines": ["citruskin bright", "페이스 크림 30g"]}
{"lines": ["citruskin bright", "아이 크림 20g"]}
{"lines": ["cloudharbor fresh", "미스트 100ml"]}
{"lines": ["cloudharbor fresh", "토너 200ml"]}
{"lines": ["freshmuse matte", "앰플 30ml"]}
{"lines": ["freshmuse matte", "수딩젤 100ml"]}
{"lines": ["barelythere glow", "샴푸 400ml"]}
{"lines": ["barelythere glow", "트리트먼트 200ml"]}
{"lines": ["softcove satin", "클레이 마스크 100g"]}
{"lines": ["softcove satin", "멀티밤 15g"]}
{"lines": ["neobare herb", "립밤 4g"]}
{"lines": ["neobare herb", "립틴트 3.5g"]}
{"lines": ["bloomix airy", "슬리핑 마스크 100g"]}
{"lines": ["bloomix airy", "페이스 크림 70g"]}
{"lines": ["ambrleaf daily", "약산성 폼 150ml"]}
{"lines": ["ambrleaf daily", "토너 150ml"]}
{"lines": ["oceandawn night", "바디 로션 150ml"]}
{"lines": ["oceandawn night", "핸드크림 50ml"]}
{"lines": ["nudemist clean", "바디워시 300ml"]}
{"lines": ["nudemist clean", "바디 로션 500ml"]}
{"lines": ["pinkharbor tint", "토너 230ml"]}
{"lines": ["pinkharbor tint", "세럼 30ml"]}
{"lines": ["quietlily soothe", "클렌징 오일 200ml"]}
{"lines": ["quietlily soothe", "클렌징 폼 120ml"]}
{"lines": ["lucidseed vita", "클렌징 워터 400ml"]}
{"lines": ["lucidseed vita", "클렌징 젤 150ml"]}
{"lines": ["miraflow aqua", "선크림 60ml"]}
{"lines": ["miraflow aqua", "토너 180ml"]}
{"lines": ["solun velvet", "선 에센스 50ml"]}
{"lines": ["solun velvet", "로션 300ml"]}
{"lines": ["aurevia mild", "페이스 크림 70g"]}
{"lines": ["aurevia mild", "아이 크림 15g"]}
{"lines": ["glaciera smooth", "미스트 50ml"]}
{"lines": ["glaciera smooth", "토너 170ml"]}
{"lines": ["fawnroot bloom", "앰플 30ml"]}
{"lines": ["fawnroot bloom", "수딩젤 150ml"]}
{"lines": ["rosenest cloud", "샴푸 500ml"]}
{"lines": ["rosenest cloud", "트리트먼트 400ml"]}
{"lines": ["opaline warm", "클레이 마스크 50g"]}
{"lines": ["opaline warm", "멀티밤 20g"]}
{"lines": ["candlea zen", "립밤 5g"]}
{"lines": ["candlea zen", "립틴트 4g"]}
{"lines": ["nortex calm", "슬리핑 마스크 100g"]}
{"lines": ["nortex calm", "페이스 크림 50g"]}
{"lines": ["silvaris dew", "약산성 폼 120ml"]}
{"lines": ["silvaris dew", "토너 200ml"]}
{"lines": ["cedarvein pure", "바디 로션 473ml"]}
{"lines": ["cedarvein pure", "핸드크림 30ml"]}
{"lines": ["mistralyn soft", "바디워시 400ml"]}
{"lines": ["mistralyn soft", "바디 로션 200ml"]}
{"lines": ["duskelle zero", "토너 150ml"]}
{"lines": ["duskelle zero", "세럼 40ml"]}
{"lines": ["purenova lift", "클렌징 오일 170ml"]}
{"lines": ["purenova lift", "클렌징 폼 100ml"]}
{"lines": ["bluecove hydra", "클렌징 워터 200ml"]}
{"lines": ["bluecove hydra", "클렌징 젤 180ml"]}
{"lines": ["honeyra cica", "선크림 50ml"]}
{"lines": ["honeyra cica", "토너 250ml"]}
{"lines": ["laventh bright", "선 에센스 50ml"]}
{"lines": ["laventh bright", "로션 150ml"]}
{"lines": ["springleaf fresh", "페이스 크림 50g"]}
{"lines": ["springleaf fresh", "아이 크림 15g"]}
{"lines": ["cocomist matte", "미스트 80ml"]}
{"lines": ["cocomist matte", "토너 150ml"]}
{"lines": ["sagehaven glow", "앰플 30ml"]}
{"lines": ["sagehaven glow", "수딩젤 200ml"]}
{"lines": ["breezara satin", "샴푸 300ml"]}
{"lines": ["breezara satin", "트리트먼트 300ml"]}
{"lines": ["ivorydew herb", "클레이 마스크 50g"]}
{"lines": ["ivorydew herb", "멀티밤 20g"]}
{"lines": ["monarchia airy", "립밤 4g"]}
{"lines": ["monarchia airy", "립틴트 3.5g"]}
{"lines": ["pebblebay daily", "슬리핑 마스크 100g"]}
{"lines": ["pebblebay daily", "페이스 크림 70g"]}
{"lines": ["tenderoot night", "약산성 폼 150ml"]}
{"lines": ["tenderoot night", "토너 150ml"]}
{"lines": ["sunwisp clean", "바디 로션 200ml"]}
{"lines": ["sunwisp clean", "핸드크림 50ml"]}
{"lines": ["rainora tint", "바디워시 500ml"]}
{"lines": ["rainora tint", "바디 로션 355ml"]}
{"lines": ["citravale soothe", "토너 170ml"]}
{"lines": ["citravale soothe", "세럼 15ml"]}
{"lines": ["veloria vita", "클렌징 오일 150ml"]}
{"lines": ["veloria vita", "클렌징 폼 150ml"]}
{"lines": ["coralyn aqua", "클렌징 워터 400ml"]}
{"lines": ["coralyn aqua", "클렌징 젤 150ml"]}
{"lines": ["mossmint velvet", "선크림 60ml"]}
{"lines": ["mossmint velvet", "토너 180ml"]}
{"lines": ["lilacove mild", "선 에센스 50ml"]}
{"lines": ["lilacove mild", "로션 200ml"]}
{"lines": ["plumaria smooth", "페이스 크림 30g"]}
{"lines": ["plumaria smooth", "아이 크림 20g"]}
{"lines": ["starling bloom", "미스트 50ml"]}
{"lines": ["starling bloom", "토너 200ml"]}
{"lines": ["mistoria cloud", "앰플 30ml"]}
{"lines": ["mistoria cloud", "수딩젤 150ml"]}
{"lines": ["seaquill warm", "샴푸 400ml"]}
{"lines": ["seaquill warm", "트리트먼트 200ml"]}
{"lines": ["dawnmuse zen", "클레이 마스크 100g"]}
{"lines": ["dawnmuse zen", "멀티밤 15g"]}
{"lines": ["aurorite calm", "립밤 5g"]}
{"lines": ["aurorite calm", "립틴트 4g"]}
{"lines": ["glowridge dew", "슬리핑 마스크 100g"]}
{"lines": ["glowridge dew", "페이스 크림 50g"]}
{"lines": ["silkbay pure", "약산성 폼 120ml"]}
{"lines": ["silkbay pure", "토너 150ml"]}
{"lines": ["cloudmori soft", "바디 로션 300ml"]}
{"lines": ["cloudmori soft", "핸드크림 100ml"]}
{"lines": ["petalfog zero", "바디워시 355ml"]}
{"lines": ["petalfog zero", "바디 로션 473ml"]}
{"lines": ["nectarune lift", "토너 230ml"]}
{"lines": ["nectarune lift", "세럼 40ml"]}
{"lines": ["halcyon hydra", "클렌징 오일 200ml"]}
{"lines": ["halcyon hydra", "클렌징 폼 100ml"]}
{"lines": ["mellowine cica", "클렌징 워터 300ml"]}
{"lines": ["mellowine cica", "클렌징 젤 180ml"]}
{"lines": ["satinelle lift", "선크림 50ml"]}
{"lines": ["satinelle lift", "토너 170ml"]}
{"lines": ["moodleaf hydra", "선 에센스 50ml"]}
{"lines": ["moodleaf hydra", "로션 150ml"]}
{"lines": ["clearhaven cica", "페이스 크림 50g"]}
{"lines": ["clearhaven cica", "아이 크림 20g"]}
{"lines": ["sproutia bright", "미스트 100ml"]}
{"lines": ["sproutia bright", "토너 180ml"]}
{"lines": ["luminote fresh", "앰플 30ml"]}
{"lines": ["luminote fresh", "수딩젤 200ml"]}
""".strip()

# 데이터 로드
raw_records = [json.loads(line) for line in RECORDS_JSONL.splitlines() if line.strip()]

# ID 재할당
records = []
for i, rec in enumerate(raw_records):
    rec['id'] = f"{i+1:05d}"
    records.append(rec)

# Pillow 리샘플링 호환성
try: RESAMPLE_LANCZOS = Image.Resampling.LANCZOS
except AttributeError: RESAMPLE_LANCZOS = Image.LANCZOS

# =========================
# Utils
# =========================
def get_clean_font_name(font_path: Path) -> str:
    stem = font_path.stem
    clean_name = re.sub(r"[^a-zA-Z0-9가-힣]", "", stem)
    return clean_name

def get_random_dark_color():
    """
    [테스트용] 색상이 적용되는지 확인하기 위해
    일부러 티가 많이 나는 색상(파랑, 빨강, 초록 등)을 반환합니다.
    """
    # 테스트를 위해 무조건 유채색 반환
    base_r = random.randint(50, 150)
    base_g = random.randint(50, 150)
    base_b = random.randint(50, 150)

    # 너무 밝아서 흰색 배경에 안 보일 경우를 대비해 하나는 어둡게
    darken = random.choice(['r', 'g', 'b'])
    if darken == 'r': base_r = random.randint(0, 50)
    elif darken == 'g': base_g = random.randint(0, 50)
    else: base_b = random.randint(0, 50)

    return (base_r, base_g, base_b)

def normalize_lines(lines):
    """
    1. 슬래시(/) 분리
    2. 용량 표기 분리
    3. [NEW] 문단 사이 빈 줄(' ') 삽입
    """
    temp_lines = []

    # 1차: 기존 정책 분리
    for line in lines:
        s = str(line).strip()
        if not s: continue
        if SLASH_POLICY == "split" and "/" in s:
            parts = [p.strip() for p in re.split(r"\s*/\s*", s) if p.strip()]
            temp_lines.extend(parts)
        elif SLASH_POLICY == "remove":
            temp_lines.append(re.sub(r"\s*/\s*", " ", s).strip())
        else:
            temp_lines.append(s)

    # 2차: 용량 패턴 분리
    parsed_lines = []
    for line in temp_lines:
        match = VOLUME_PATTERN.match(line)
        if match:
            main_part = match.group(1).strip()
            vol_part = match.group(2).strip()
            if main_part: parsed_lines.append(main_part)
            parsed_lines.append(vol_part)
        else:
            parsed_lines.append(line)

    # 3차: 빈 줄 삽입 (Double Spacing)
    if ADD_EMPTY_LINE and len(parsed_lines) > 0:
        spaced_lines = []
        for i, item in enumerate(parsed_lines):
            spaced_lines.append(item)
            # 마지막 아이템이 아니면 공백(" ") 추가 -> 이게 빈 줄 역할
            if i < len(parsed_lines) - 1:
                spaced_lines.append(" ")
        return spaced_lines

    return parsed_lines

def best_cmap(font_path: Path) -> dict:
    try:
        tt = TTFont(str(font_path), lazy=True)
        cmap = tt["cmap"].getBestCmap() or {}
        try: tt.close()
        except: pass
        return cmap
    except: return {}

def missing_chars(cmap: dict, text: str):
    miss = []
    for ch in set(text):
        if ch.isspace(): continue
        if ord(ch) not in cmap: miss.append(ch)
    return sorted(miss)

def tight_crop_white(img: Image.Image) -> Image.Image:
    if not PIXEL_CROP: return img
    bg = Image.new("RGB", img.size, (255, 255, 255))
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()
    return img.crop(bbox) if bbox else img

def fit_into_square(img: Image.Image, size: int) -> tuple[Image.Image, float, tuple[int,int]]:
    w, h = img.size
    scale = min(size / w, size / h, 1.0)
    if scale < 1.0:
        nw = max(1, int(round(w * scale)))
        nh = max(1, int(round(h * scale)))
        img = img.resize((nw, nh), resample=RESAMPLE_LANCZOS)
        w, h = img.size
    canvas = Image.new("RGB", (size, size), (255, 255, 255))
    ox = (size - w) // 2
    oy = (size - h) // 2
    canvas.paste(img, (ox, oy))
    return canvas, float(scale), (int(ox), int(oy))

def get_text_dimensions(text_string, font):
    dummy = Image.new("RGB", (1, 1))
    draw = ImageDraw.Draw(dummy)
    if hasattr(draw, "textbbox"):
        l, t, r, b = draw.textbbox((0, 0), text_string, font=font)
        return r - l, b - t
    else:
        return draw.textsize(text_string, font)

# =========================
# 렌더링 함수 (랜덤 색상 적용)
# =========================
def render_multi_size_block(lines, font_path: Path): # -> tuple[Image.Image, tuple]
    font_main = ImageFont.truetype(str(font_path), RENDER_FONT_SIZE)
    font_small = ImageFont.truetype(str(font_path), int(RENDER_FONT_SIZE * SMALL_FONT_RATIO))

    # ⭐ 랜덤 색상 생성
    text_color = get_random_dark_color()

    parsed_lines = []
    total_h = 0
    max_w = 0

    for line in lines:
        # 공백 라인(" ")은 정규식 매치 X -> font_main 사용 -> 빈 줄 높이 확보
        is_vol = bool(VOLUME_PATTERN.match(line))
        font = font_small if is_vol else font_main

        w, h = get_text_dimensions(line, font)

        # 만약 " " 공백 줄이라면 높이는 폰트 크기만큼 보장, 너비는 0에 가까움
        if not line.strip():
            h = RENDER_FONT_SIZE # 강제 높이 지정

        parsed_lines.append({"text": line, "font": font, "w": w, "h": h})
        max_w = max(max_w, w)
        total_h += h

    # 줄 간격 추가
    total_h += (len(lines) - 1) * LINE_GAP if len(lines) > 0 else 0

    img_w = max_w + 2 * PAD
    img_h = total_h + 2 * PAD
    img = Image.new("RGB", (img_w, img_h), (255, 255, 255))
    draw = ImageDraw.Draw(img)

    current_y = PAD
    for item in parsed_lines:
        text = item["text"]
        font = item["font"]
        line_w = item["w"]
        line_h = item["h"]

        x = (img_w - line_w) // 2

        # 공백이 아닌 경우만 실제 그리기
        if text.strip():
            # ⭐ fill 옵션에 랜덤 색상 적용
            draw.text((x, current_y), text, font=font, fill=text_color)

        current_y += line_h + LINE_GAP

    # 이미지와 사용된 색상 반환
    return img, text_color

# =========================
# 메인 프로세스
# =========================
def process_single_font_safe(font_path, record_list):
    clean_font_name = get_clean_font_name(font_path) or "UnknownFont"
    out_dir = OUT_ROOT / clean_font_name

    ann_path  = out_dir / "annotations.jsonl"
    if ann_path.exists():
        print(f"⏩ Skipping {clean_font_name}", flush=True)
        return

    cmap = best_cmap(font_path)
    if not cmap:
        print(f"⚠️ Bad font: {clean_font_name}", flush=True)
        return

    out_dir.mkdir(parents=True, exist_ok=True)
    miss_path = out_dir / "missing_glyphs.jsonl"
    saved, skipped = 0, 0

    with ann_path.open("w", encoding="utf-8") as ann_f, miss_path.open("w", encoding="utf-8") as miss_f:
        for rec in record_list:
            rid = rec.get("id")
            raw_lines = rec.get("lines", [])
            lines = normalize_lines(raw_lines)
            text = "\n".join(lines)

            miss = missing_chars(cmap, text)
            if miss:
                miss_f.write(json.dumps({"id": rid, "font": str(font_path), "missing_chars": miss}, ensure_ascii=False) + "\n")
                if SKIP_MISSING_GLYPHS:
                    skipped += 1
                    continue

            try:
                # 렌더링 호출 (이미지와 색상 정보 받음)
                img, used_text_color = render_multi_size_block(lines, font_path)
            except Exception:
                continue

            img = tight_crop_white(img)
            inner_w, inner_h = img.size
            final_img, scale, (ox, oy) = fit_into_square(img, OUTPUT_SIZE)

            filename = f"{clean_font_name}_{rid}.png"
            final_img.save(out_dir / filename, "PNG")

            meta = {
                "id": rid,
                "file_name": filename,
                "image_path": f"{clean_font_name}/{filename}",
                "font_name": clean_font_name,
                "output_size": OUTPUT_SIZE,
                "render_font_size": RENDER_FONT_SIZE,
                "small_font_ratio": SMALL_FONT_RATIO,
                "fit_scale": float(scale),
                "paste_offset": [ox, oy],
                "text_color_rgb": used_text_color, # ✅ 사용된 색상 정보 저장
                "lines": lines
            }
            ann_f.write(json.dumps(meta, ensure_ascii=False) + "\n")
            saved += 1

    print(f"[{clean_font_name}] Saved: {saved}, Skipped: {skipped}", flush=True)

if __name__ == "__main__":
    OUT_ROOT.mkdir(parents=True, exist_ok=True)
    ttf_paths = sorted([p for p in FONTS_DIR.rglob("*.ttf") if p.is_file()])

    print(f"Records: {len(records)}, Fonts: {len(ttf_paths)}")
    print("--------------------------------------------------")
    print("🎨 [Smart Color Mode] Generating Front Labels with Random Dark Colors...")
    print("--------------------------------------------------")
    for i, font_path in enumerate(ttf_paths):
        print(f"👉 [{i+1}/{len(ttf_paths)}] {font_path.name} ... ", end="", flush=True)
        try: process_single_font_safe(font_path, records)
        except KeyboardInterrupt: break
        except Exception as e: print(f"\n❌ Error: {e}", flush=True)

    print("\n✅ All Done.")

Records: 200, Fonts: 150
--------------------------------------------------
🎨 [Smart Color Mode] Generating Front Labels with Random Dark Colors...
--------------------------------------------------
👉 [1/150] AritaBuriKR-Medium.ttf ... [AritaBuriKRMedium] Saved: 200, Skipped: 0
👉 [2/150] AstaSans-Regular.ttf ... [AstaSansRegular] Saved: 200, Skipped: 0
👉 [3/150] Binggrae.ttf ... [Binggrae] Saved: 200, Skipped: 0
👉 [4/150] BinggraeII.ttf ... [BinggraeII] Saved: 200, Skipped: 0
👉 [5/150] BinggraeMelona.ttf ... [BinggraeMelona] Saved: 200, Skipped: 0
👉 [6/150] BinggraeSamanco.ttf ... [BinggraeSamanco] Saved: 200, Skipped: 0
👉 [7/150] BinggraeTaom.ttf ... [BinggraeTaom] Saved: 200, Skipped: 0
👉 [8/150] BMHANNA_11yrs_ttf.ttf ... [BMHANNA11yrsttf] Saved: 200, Skipped: 0
👉 [9/150] BookkGothic_Light.ttf ... [BookkGothicLight] Saved: 200, Skipped: 0
👉 [10/150] BookkMyungjo_Light.ttf ... [BookkMyungjoLight] Saved: 200, Skipped: 0
👉 [11/150] Cafe24Simplehae-v2.0.ttf ... [Cafe24Simplehaev20] Saved

103736 extra bytes in post.stringData array


[SpoqaHanSansRegular] Saved: 200, Skipped: 0
👉 [107/150] SpoqaHanSansNeo-Regular.ttf ... [SpoqaHanSansNeoRegular] Saved: 200, Skipped: 0
👉 [108/150] STUNNING.ttf ... [STUNNING] Saved: 200, Skipped: 0
👉 [109/150] SUIT-Variable.ttf ... [SUITVariable] Saved: 200, Skipped: 0
👉 [110/150] The Jamsil 3 Regular.ttf ... [TheJamsil3Regular] Saved: 200, Skipped: 0
👉 [111/150] THEFACESHOP+INKLIPQUID(윈도우용).ttf ... [THEFACESHOPINKLIPQUID윈도우용] Saved: 200, Skipped: 0
👉 [112/150] Tlab신영복체.ttf ... [Tlab신영복체] Saved: 200, Skipped: 0
👉 [113/150] UhBee Se_hyun.ttf ... [UhBeeSehyun] Saved: 200, Skipped: 0
👉 [114/150] WantedSans-Regular.ttf ... [WantedSansRegular] Saved: 200, Skipped: 0
👉 [115/150] WantedSansVariable.ttf ... [WantedSansVariable] Saved: 200, Skipped: 0
👉 [116/150] YES24GothicR.ttf ... [YES24GothicR] Saved: 200, Skipped: 0
👉 [117/150] YES24MyoungjoR.ttf ... [YES24MyoungjoR] Saved: 200, Skipped: 0
👉 [118/150] YoonChildfundkoreaMinGuk.ttf ... [YoonChildfundkoreaMinGuk] Saved: 200, Skipped: 0
👉 [1

12292 extra bytes in post.stringData array


[중나좋체Medium] Saved: 200, Skipped: 0
👉 [150/150] 클립아트코리아_Regular.ttf ... [클립아트코리아Regular] Saved: 200, Skipped: 0

✅ All Done.


## 가상 화장품 뒷면에 들어갈 폰트 이미지 생성하기

In [6]:
import pandas as pd
import json
import re
import sys
import textwrap  # 줄바꿈 처리를 위한 모듈
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont, ImageChops
from fontTools.ttLib import TTFont

# =========================
# ✅ 1. 설정 및 경로
# =========================
FONTS_DIR  = Path(r"C:\Users\nam\Desktop\tst\ref_fonts_ttf")
OUT_ROOT   = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\BackPage")
CSV_PATH   = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\back_labels_200_fixed_21lines_max34.csv")

# 렌더링 옵션 (뒷면 라벨용)
RENDER_FONT_SIZE = 42           # 작은 글씨
LINE_SPACING = 10               # 줄 간격 (픽셀)
IMAGE_WIDTH = 1024              # 이미지 가로 크기
IMAGE_HEIGHT = 1024             # 이미지 세로 크기
PADDING_X = 50                  # 좌우 여백
PADDING_Y = 50                  # 상하 여백
MAX_CHARS_PER_LINE = 35         # 한 줄당 최대 글자 수 (폰트 크기에 따라 조절 필요)

SKIP_MISSING_GLYPHS = True

# =========================
# ✅ 2. 데이터 로드 (CSV)
# =========================
# CSV 파일을 읽어서 리스트로 변환합니다.
try:
    # 제공된 파일명과 구조를 기반으로 읽기 (헤더가 없거나 'text' 컬럼이 있다고 가정)
    # 실제 파일에 헤더가 없다면 names=['text'], header=None 사용
    df = pd.read_csv(CSV_PATH)
    # 텍스트 컬럼이 'text'라고 가정, 없다면 첫 번째 컬럼 사용
    col_name = 'text' if 'text' in df.columns else df.columns[0]
    back_label_texts = df[col_name].dropna().astype(str).tolist()

    print(f"Loaded {len(back_label_texts)} back label records.")

except Exception as e:
    print(f"❌ CSV Load Error: {e}")
    # 테스트용 더미 데이터 (파일 없을 때 실행용)
    back_label_texts = [
        "사용방법\n적당량을 덜어 얼굴에 펴 바르세요.\n\n전성분\n정제수, 글리세린, 부틸렌글라이콜",
        "주의사항\n1. 붉은 반점 등이 나타나면 사용 중단.\n2. 상처 부위 사용 금지."
    ] * 100

# 데이터 개수 맞추기 (200개)
back_label_texts = back_label_texts[:200]

# =========================
# Utils
# =========================
def get_clean_font_name(font_path: Path) -> str:
    stem = font_path.stem
    clean_name = re.sub(r"[^a-zA-Z0-9가-힣]", "", stem)
    return clean_name

def best_cmap(font_path: Path) -> dict:
    try:
        tt = TTFont(str(font_path), lazy=True)
        cmap = tt["cmap"].getBestCmap() or {}
        try: tt.close()
        except: pass
        return cmap
    except: return {}

def missing_chars(cmap: dict, text: str):
    miss = []
    for ch in set(text):
        if ch.isspace(): continue
        if ord(ch) not in cmap: miss.append(ch)
    return sorted(miss)

# =========================
# ⭐ [핵심] 뒷면 라벨 렌더링 (줄바꿈 + 좌측 정렬)
# =========================
def render_back_label(full_text, font_path: Path) -> tuple[Image.Image, list]:
    font = ImageFont.truetype(str(font_path), RENDER_FONT_SIZE)

    # 1. 텍스트 줄바꿈 처리 (Text Wrapping)
    # 원본 텍스트의 엔터(\n)는 유지하되, 너무 긴 줄은 자릅니다.
    wrapped_lines = []

    # 원본 텍스트를 줄바꿈 기준으로 나눔
    paragraphs = full_text.split('\n')

    for paragraph in paragraphs:
        if not paragraph.strip():
            wrapped_lines.append("") # 빈 줄 유지
            continue
        # 긴 문장을 설정된 너비로 자름
        wrapped = textwrap.wrap(paragraph, width=MAX_CHARS_PER_LINE)
        wrapped_lines.extend(wrapped)

    # 2. 캔버스 높이 계산
    # (글자 높이 + 줄간격) * 줄 수
    # 정확한 높이 계산을 위해 bbox 사용
    dummy_draw = ImageDraw.Draw(Image.new("RGB", (1, 1)))
    line_heights = []
    max_line_w = 0

    for line in wrapped_lines:
        if not line:
            # 빈 줄은 폰트 크기의 50% 정도 높이로 설정
            line_heights.append(int(RENDER_FONT_SIZE * 0.5))
            continue

        if hasattr(dummy_draw, "textbbox"):
            bbox = dummy_draw.textbbox((0, 0), line, font=font)
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
        else:
            w, h = dummy_draw.textsize(line, font=font)

        line_heights.append(h)
        max_line_w = max(max_line_w, w)

    # 전체 텍스트 높이
    total_text_h = sum(line_heights) + (len(wrapped_lines) - 1) * LINE_SPACING

    # 캔버스 생성 (1024x1024 고정 or 내용에 맞게? -> 여기선 1024 Fit을 위해 넉넉히 생성 후 리사이즈)
    # 일단 내용에 딱 맞는 캔버스 생성 후 나중에 Fit
    canvas_w = max_line_w + (PADDING_X * 2)
    canvas_h = total_text_h + (PADDING_Y * 2)

    img = Image.new("RGB", (canvas_w, canvas_h), (255, 255, 255))
    draw = ImageDraw.Draw(img)

    # 3. 그리기
    current_y = PADDING_Y
    for i, line in enumerate(wrapped_lines):
        h = line_heights[i]
        if line:
            draw.text((PADDING_X, current_y), line, font=font, fill=(0, 0, 0))
        current_y += h + LINE_SPACING

    return img, wrapped_lines

def fit_into_square(img: Image.Image, size: int) -> tuple[Image.Image, float, tuple[int,int]]:
    # 이미지를 1024x1024 안에 비율 유지하며 넣기
    w, h = img.size
    scale = min(size / w, size / h, 1.0) # 확대는 안 함 (1.0 cap)

    nw = int(w * scale)
    nh = int(h * scale)

    img_resized = img.resize((nw, nh), resample=Image.Resampling.LANCZOS)

    canvas = Image.new("RGB", (size, size), (255, 255, 255))
    # 중앙 정렬
    ox = (size - nw) // 2
    oy = (size - nh) // 2
    canvas.paste(img_resized, (ox, oy))

    return canvas, scale, (ox, oy)

# =========================
# 메인 프로세스
# =========================
def process_single_font_back(font_path, text_list):
    clean_font_name = get_clean_font_name(font_path) or "UnknownFont"
    out_dir = OUT_ROOT / clean_font_name

    ann_path  = out_dir / "annotations.jsonl"
    if ann_path.exists():
        print(f"⏩ Skipping {clean_font_name} (Already done)", flush=True)
        return

    cmap = best_cmap(font_path)
    if not cmap:
        print(f"⚠️ Bad font: {clean_font_name}", flush=True)
        return

    out_dir.mkdir(parents=True, exist_ok=True)
    miss_path = out_dir / "missing_glyphs.jsonl"
    saved, skipped = 0, 0

    with ann_path.open("w", encoding="utf-8") as ann_f, miss_path.open("w", encoding="utf-8") as miss_f:
        for idx, text in enumerate(text_list):
            rid = f"{idx+1:05d}" # ID: 00001 ~ 00200

            # 글자 누락 확인
            miss = missing_chars(cmap, text)
            if miss:
                miss_f.write(json.dumps({"id": rid, "font": str(font_path), "missing_chars": miss}, ensure_ascii=False) + "\n")
                if SKIP_MISSING_GLYPHS:
                    skipped += 1
                    continue

            try:
                # 렌더링
                img, processed_lines = render_back_label(text, font_path)

                # 1024x1024 중앙 배치
                final_img, scale, (ox, oy) = fit_into_square(img, IMAGE_HEIGHT)

                # 저장
                filename = f"{clean_font_name}_back_{rid}.png"
                final_img.save(out_dir / filename, "PNG")

                # 메타데이터
                meta = {
                    "id": rid,
                    "file_name": filename,
                    "image_path": f"{clean_font_name}/{filename}",
                    "font_name": clean_font_name,
                    "type": "back_label",
                    "original_text": text,
                    "processed_lines": processed_lines, # 줄바꿈된 최종 텍스트
                    "fit_scale": float(scale),
                    "paste_offset": [ox, oy]
                }
                ann_f.write(json.dumps(meta, ensure_ascii=False) + "\n")
                saved += 1

            except Exception as e:
                # print(f"Error on {rid}: {e}")
                continue

    print(f"[{clean_font_name}] Saved: {saved}, Skipped: {skipped}", flush=True)

if __name__ == "__main__":
    OUT_ROOT.mkdir(parents=True, exist_ok=True)

    ttf_paths = sorted([p for p in FONTS_DIR.rglob("*.ttf") if p.is_file()])

    print(f"Back Labels: {len(back_label_texts)}, Fonts: {len(ttf_paths)}")
    print("--------------------------------------------------")

    for i, font_path in enumerate(ttf_paths):
        print(f"👉 [{i+1}/{len(ttf_paths)}] {font_path.name} ... ", end="", flush=True)
        try: process_single_font_back(font_path, back_label_texts)
        except KeyboardInterrupt: break
        except Exception as e: print(f"\n❌ Error: {e}", flush=True)

    print("\n✅ All Done.")

Loaded 200 back label records.
Back Labels: 200, Fonts: 150
--------------------------------------------------
👉 [1/150] AritaBuriKR-Medium.ttf ... [AritaBuriKRMedium] Saved: 200, Skipped: 0
👉 [2/150] AstaSans-Regular.ttf ... [AstaSansRegular] Saved: 200, Skipped: 0
👉 [3/150] Binggrae.ttf ... [Binggrae] Saved: 200, Skipped: 0
👉 [4/150] BinggraeII.ttf ... [BinggraeII] Saved: 200, Skipped: 0
👉 [5/150] BinggraeMelona.ttf ... [BinggraeMelona] Saved: 200, Skipped: 0
👉 [6/150] BinggraeSamanco.ttf ... [BinggraeSamanco] Saved: 200, Skipped: 0
👉 [7/150] BinggraeTaom.ttf ... [BinggraeTaom] Saved: 200, Skipped: 0
👉 [8/150] BMHANNA_11yrs_ttf.ttf ... [BMHANNA11yrsttf] Saved: 200, Skipped: 0
👉 [9/150] BookkGothic_Light.ttf ... [BookkGothicLight] Saved: 200, Skipped: 0
👉 [10/150] BookkMyungjo_Light.ttf ... [BookkMyungjoLight] Saved: 200, Skipped: 0
👉 [11/150] Cafe24Simplehae-v2.0.ttf ... [Cafe24Simplehaev20] Saved: 0, Skipped: 200
👉 [12/150] Cafe24SsurroundAir-v1.1.ttf ... [Cafe24SsurroundAirv11] Sav

103736 extra bytes in post.stringData array


[SpoqaHanSansRegular] Saved: 200, Skipped: 0
👉 [107/150] SpoqaHanSansNeo-Regular.ttf ... [SpoqaHanSansNeoRegular] Saved: 200, Skipped: 0
👉 [108/150] STUNNING.ttf ... [STUNNING] Saved: 200, Skipped: 0
👉 [109/150] SUIT-Variable.ttf ... [SUITVariable] Saved: 200, Skipped: 0
👉 [110/150] The Jamsil 3 Regular.ttf ... [TheJamsil3Regular] Saved: 200, Skipped: 0
👉 [111/150] THEFACESHOP+INKLIPQUID(윈도우용).ttf ... [THEFACESHOPINKLIPQUID윈도우용] Saved: 200, Skipped: 0
👉 [112/150] Tlab신영복체.ttf ... [Tlab신영복체] Saved: 200, Skipped: 0
👉 [113/150] UhBee Se_hyun.ttf ... [UhBeeSehyun] Saved: 0, Skipped: 200
👉 [114/150] WantedSans-Regular.ttf ... [WantedSansRegular] Saved: 200, Skipped: 0
👉 [115/150] WantedSansVariable.ttf ... [WantedSansVariable] Saved: 200, Skipped: 0
👉 [116/150] YES24GothicR.ttf ... [YES24GothicR] Saved: 200, Skipped: 0
👉 [117/150] YES24MyoungjoR.ttf ... [YES24MyoungjoR] Saved: 200, Skipped: 0
👉 [118/150] YoonChildfundkoreaMinGuk.ttf ... [YoonChildfundkoreaMinGuk] Saved: 200, Skipped: 0
👉 [1

12292 extra bytes in post.stringData array


[중나좋체Medium] Saved: 200, Skipped: 0
👉 [150/150] 클립아트코리아_Regular.ttf ... [클립아트코리아Regular] Saved: 200, Skipped: 0

✅ All Done.


## 색상을 다양하게 생성

In [9]:
import pandas as pd
import json
import re
import sys
import random
import textwrap
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from fontTools.ttLib import TTFont

# =========================
# ✅ 1. 설정 및 경로
# =========================
# 사용자 환경에 맞게 경로를 수정하세요.
FONTS_DIR  = Path(r"C:\Users\nam\Desktop\tst\ref_fonts_ttf")
OUT_ROOT   = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\back_color_aug") # 컬러 버전 폴더
CSV_PATH   = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\back_labels_200_fixed_21lines_max34.csv") # 업로드된 파일명 사용

# 렌더링 옵션
RENDER_FONT_SIZE = 42
LINE_SPACING = 10
IMAGE_WIDTH = 1024
IMAGE_HEIGHT = 1024
PADDING_X = 50
PADDING_Y = 50
MAX_CHARS_PER_LINE = 35

SKIP_MISSING_GLYPHS = True

# =========================
# ✅ 2. [수정됨] Front Label 방식의 랜덤 색상 생성 함수
# =========================
def get_random_dark_color():
    """
    [변경됨] Front Label 코드와 동일한 로직.
    무조건 검정/회색만 나오는 것을 방지하고,
    가독성이 있으면서도 다양한 유채색(파랑, 빨강, 초록, 보라 등 어두운 톤)을 생성합니다.
    """
    # 1. 기본 베이스 색상을 약간 밝게 잡습니다 (50~150)
    base_r = random.randint(50, 150)
    base_g = random.randint(50, 150)
    base_b = random.randint(50, 150)

    # 2. 흰 배경 가독성을 위해 RGB 중 하나를 강제로 어둡게 만듭니다 (0~50)
    #    이렇게 하면 '진한 파랑', '진한 초록', '진한 와인색' 등이 잘 나옵니다.
    darken = random.choice(['r', 'g', 'b'])

    if darken == 'r':
        base_r = random.randint(0, 50)
    elif darken == 'g':
        base_g = random.randint(0, 50)
    else:
        base_b = random.randint(0, 50)

    return (base_r, base_g, base_b)

# =========================
# 3. 데이터 로드 및 유틸리티
# =========================
try:
    df = pd.read_csv(CSV_PATH)
    col_name = 'text' if 'text' in df.columns else df.columns[0]
    back_label_texts = df[col_name].dropna().astype(str).tolist()
    # 200개만 사용
    back_label_texts = back_label_texts[:200]
    print(f"Loaded {len(back_label_texts)} records.")
except Exception as e:
    print(f"❌ CSV Load Error: {e}")
    back_label_texts = []

def get_clean_font_name(font_path: Path) -> str:
    return re.sub(r"[^a-zA-Z0-9가-힣]", "", font_path.stem)

def best_cmap(font_path: Path) -> dict:
    try:
        tt = TTFont(str(font_path), lazy=True)
        cmap = tt["cmap"].getBestCmap() or {}
        try: tt.close()
        except: pass
        return cmap
    except: return {}

def missing_chars(cmap: dict, text: str):
    miss = []
    for ch in set(text):
        if ch.isspace(): continue
        if ord(ch) not in cmap: miss.append(ch)
    return sorted(miss)

# =========================
# 4. 렌더링 함수 (랜덤 색상 적용)
# =========================
def render_back_label_color(full_text, font_path: Path) -> tuple[Image.Image, list, tuple]:
    font = ImageFont.truetype(str(font_path), RENDER_FONT_SIZE)

    # ⭐ (1) 텍스트 색상 결정 (수정된 함수 사용)
    text_color = get_random_dark_color()

    # (2) 줄바꿈 처리
    wrapped_lines = []
    paragraphs = full_text.split('\n')
    for paragraph in paragraphs:
        if not paragraph.strip():
            wrapped_lines.append("")
            continue
        wrapped = textwrap.wrap(paragraph, width=MAX_CHARS_PER_LINE)
        wrapped_lines.extend(wrapped)

    # (3) 높이 계산
    dummy_draw = ImageDraw.Draw(Image.new("RGB", (1, 1)))
    line_heights = []
    max_line_w = 0

    for line in wrapped_lines:
        if not line:
            line_heights.append(int(RENDER_FONT_SIZE * 0.5))
            continue
        if hasattr(dummy_draw, "textbbox"):
            bbox = dummy_draw.textbbox((0, 0), line, font=font)
            w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        else:
            w, h = dummy_draw.textsize(line, font=font)
        line_heights.append(h)
        max_line_w = max(max_line_w, w)

    total_text_h = sum(line_heights) + (len(wrapped_lines) - 1) * LINE_SPACING

    canvas_w = max_line_w + (PADDING_X * 2)
    canvas_h = total_text_h + (PADDING_Y * 2)

    img = Image.new("RGB", (canvas_w, canvas_h), (255, 255, 255))
    draw = ImageDraw.Draw(img)

    # (4) 그리기 (결정된 색상 사용)
    current_y = PADDING_Y
    for i, line in enumerate(wrapped_lines):
        h = line_heights[i]
        if line:
            # ⭐ text_color 적용
            draw.text((PADDING_X, current_y), line, font=font, fill=text_color)
        current_y += h + LINE_SPACING

    return img, wrapped_lines, text_color

def fit_into_square(img: Image.Image, size: int) -> tuple[Image.Image, float, tuple[int,int]]:
    w, h = img.size
    scale = min(size / w, size / h, 1.0)
    nw, nh = int(w * scale), int(h * scale)
    img_resized = img.resize((nw, nh), resample=Image.Resampling.LANCZOS)
    canvas = Image.new("RGB", (size, size), (255, 255, 255))
    ox, oy = (size - nw) // 2, (size - nh) // 2
    canvas.paste(img_resized, (ox, oy))
    return canvas, scale, (ox, oy)

# =========================
# 5. 메인 실행 로직
# =========================
def process_single_font_color(font_path, text_list):
    clean_font_name = get_clean_font_name(font_path) or "UnknownFont"
    out_dir = OUT_ROOT / clean_font_name

    ann_path  = out_dir / "annotations.jsonl"
    if ann_path.exists():
        print(f"⏩ Skipping {clean_font_name} (Already done)", flush=True)
        return

    cmap = best_cmap(font_path)
    if not cmap:
        print(f"⚠️ Bad font: {clean_font_name}", flush=True)
        return

    out_dir.mkdir(parents=True, exist_ok=True)
    miss_path = out_dir / "missing_glyphs.jsonl"
    saved, skipped = 0, 0

    with ann_path.open("w", encoding="utf-8") as ann_f, miss_path.open("w", encoding="utf-8") as miss_f:
        for idx, text in enumerate(text_list):
            rid = f"{idx+1:05d}"

            miss = missing_chars(cmap, text)
            if miss:
                miss_f.write(json.dumps({"id": rid, "font": str(font_path), "missing_chars": miss}, ensure_ascii=False) + "\n")
                if SKIP_MISSING_GLYPHS:
                    skipped += 1
                    continue

            try:
                # 렌더링 호출 (색상 정보도 반환받음)
                img, processed_lines, used_color = render_back_label_color(text, font_path)

                final_img, scale, (ox, oy) = fit_into_square(img, IMAGE_HEIGHT)

                filename = f"{clean_font_name}_back_{rid}.png"
                final_img.save(out_dir / filename, "PNG")

                meta = {
                    "id": rid,
                    "file_name": filename,
                    "image_path": f"{clean_font_name}/{filename}",
                    "font_name": clean_font_name,
                    "type": "back_label",
                    "original_text": text,
                    "processed_lines": processed_lines,
                    "text_color_rgb": used_color, # 사용된 색상 정보 저장
                    "fit_scale": float(scale),
                    "paste_offset": [ox, oy]
                }
                ann_f.write(json.dumps(meta, ensure_ascii=False) + "\n")
                saved += 1

            except Exception:
                continue

    print(f"[{clean_font_name}] Saved: {saved}, Skipped: {skipped}", flush=True)

if __name__ == "__main__":
    # 경로 생성
    OUT_ROOT.mkdir(parents=True, exist_ok=True)

    # 폰트 찾기
    ttf_paths = sorted([p for p in FONTS_DIR.rglob("*.ttf") if p.is_file()])

    print(f"Data Records: {len(back_label_texts)}")
    print(f"Fonts Found: {len(ttf_paths)}")
    print("--------------------------------------------------")
    print("🎨 [Diverse Color Mode] Generating Back Labels with Random Colors...")
    print("--------------------------------------------------")

    for i, font_path in enumerate(ttf_paths):
        print(f"👉 [{i+1}/{len(ttf_paths)}] {font_path.name} ... ", end="", flush=True)
        try:
            process_single_font_color(font_path, back_label_texts)
        except KeyboardInterrupt:
            print("\n🛑 Stopped by user.")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}", flush=True)

    print("\n✅ All Done.")

Loaded 200 records.
Data Records: 200
Fonts Found: 150
--------------------------------------------------
🎨 [Diverse Color Mode] Generating Back Labels with Random Colors...
--------------------------------------------------
👉 [1/150] AritaBuriKR-Medium.ttf ... [AritaBuriKRMedium] Saved: 200, Skipped: 0
👉 [2/150] AstaSans-Regular.ttf ... [AstaSansRegular] Saved: 200, Skipped: 0
👉 [3/150] Binggrae.ttf ... [Binggrae] Saved: 200, Skipped: 0
👉 [4/150] BinggraeII.ttf ... [BinggraeII] Saved: 200, Skipped: 0
👉 [5/150] BinggraeMelona.ttf ... [BinggraeMelona] Saved: 200, Skipped: 0
👉 [6/150] BinggraeSamanco.ttf ... [BinggraeSamanco] Saved: 200, Skipped: 0
👉 [7/150] BinggraeTaom.ttf ... [BinggraeTaom] Saved: 200, Skipped: 0
👉 [8/150] BMHANNA_11yrs_ttf.ttf ... [BMHANNA11yrsttf] Saved: 200, Skipped: 0
👉 [9/150] BookkGothic_Light.ttf ... [BookkGothicLight] Saved: 200, Skipped: 0
👉 [10/150] BookkMyungjo_Light.ttf ... [BookkMyungjoLight] Saved: 200, Skipped: 0
👉 [11/150] Cafe24Simplehae-v2.0.ttf ... 

103736 extra bytes in post.stringData array


[SpoqaHanSansRegular] Saved: 200, Skipped: 0
👉 [107/150] SpoqaHanSansNeo-Regular.ttf ... [SpoqaHanSansNeoRegular] Saved: 200, Skipped: 0
👉 [108/150] STUNNING.ttf ... [STUNNING] Saved: 200, Skipped: 0
👉 [109/150] SUIT-Variable.ttf ... [SUITVariable] Saved: 200, Skipped: 0
👉 [110/150] The Jamsil 3 Regular.ttf ... [TheJamsil3Regular] Saved: 200, Skipped: 0
👉 [111/150] THEFACESHOP+INKLIPQUID(윈도우용).ttf ... [THEFACESHOPINKLIPQUID윈도우용] Saved: 200, Skipped: 0
👉 [112/150] Tlab신영복체.ttf ... [Tlab신영복체] Saved: 200, Skipped: 0
👉 [113/150] UhBee Se_hyun.ttf ... [UhBeeSehyun] Saved: 0, Skipped: 200
👉 [114/150] WantedSans-Regular.ttf ... [WantedSansRegular] Saved: 200, Skipped: 0
👉 [115/150] WantedSansVariable.ttf ... [WantedSansVariable] Saved: 200, Skipped: 0
👉 [116/150] YES24GothicR.ttf ... [YES24GothicR] Saved: 200, Skipped: 0
👉 [117/150] YES24MyoungjoR.ttf ... [YES24MyoungjoR] Saved: 200, Skipped: 0
👉 [118/150] YoonChildfundkoreaMinGuk.ttf ... [YoonChildfundkoreaMinGuk] Saved: 200, Skipped: 0
👉 [1

12292 extra bytes in post.stringData array


[중나좋체Medium] Saved: 200, Skipped: 0
👉 [150/150] 클립아트코리아_Regular.ttf ... [클립아트코리아Regular] Saved: 200, Skipped: 0

✅ All Done.


In [ ]:
# ❌ Skipped 폰트 목록 (총 24개)
#
# 1. 나눔손글씨 시리즈 (8개) 손글씨 폰트가 대거 스킵되었습니다.
#
#     [124] 나눔손글씨 곰신체.ttf
#
#     [125] 나눔손글씨 맛있는체.ttf
#
#     [126] 나눔손글씨 상해찬미체.ttf
#
#     [127] 나눔손글씨 성실체.ttf
#
#     [128] 나눔손글씨 소방관의 기도.ttf
#
#     [129] 나눔손글씨 아인맘 손글씨.ttf
#
#     [130] 나눔손글씨 엄마사랑.ttf
#
#     [131] 나눔손글씨 예쁜 민경체.ttf
#
#     [132] 나눔손글씨 의미있는 한글.ttf
#
#     [133] 나눔손글씨 철필글씨.ttf
#
# 2. 온글잎 시리즈 (5개) 마찬가지로 손글씨 스타일입니다.
#
#     [139] 온글잎 공부잘하자나.ttf
#
#     [140] 온글잎 김콩해.ttf
#
#     [141] 온글잎 바닷바람.ttf
#
#     [142] 온글잎 위씨리스트.ttf
#
#     [143] 온글잎 콘콘체.ttf
#
# 3. MinSans 시리즈 (3개)
#
#     [77] MinSans-Light.ttf
#
#     [78] MinSans-Regular.ttf
#
#     [79] MinSansVF.ttf (가변 폰트)
#
# 4. Cafe24 시리즈 (2개)
#
#     [11] Cafe24Simplehae-v2.0.ttf
#
#     [12] Cafe24SsurroundAir-v1.1.ttf
#
# 5. Hahmlet (함렛) 시리즈 (2개)
#
#     [36] Hahmlet-Black.ttf
#
#     [37] Hahmlet-Thin.ttf
#
# 6. 기타 (4개)
#
#     [75] MapoHongdaeFreedom.ttf (마포홍대프리덤)
#
#     [113] UhBee Se_hyun.ttf (어비 세현 - 손글씨)

## 24개 폰트 (특히 손글씨 관련 폰트가 뒷면 데이터셋 생성에 실패)
## >> 126개로 진행

In [10]:
import shutil
import re
from pathlib import Path

# =========================
# ✅ 1. 설정 및 경로
# =========================
DATASET_ROOT = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋")
FAILED_DATASET_ROOT = Path(r"C:\Users\nam\Desktop\_Failed_")
TARGET_SUBFOLDERS = ["back_bw", "back_color_aug", "front_bw", "front_color_aug"]

# 📝 로그에서 추출한 실패 폰트 목록 (24개)
FAILED_FONT_FILENAMES = [
    # 1. 나눔손글씨
    "나눔손글씨 곰신체.ttf", "나눔손글씨 맛있는체.ttf", "나눔손글씨 상해찬미체.ttf",
    "나눔손글씨 성실체.ttf", "나눔손글씨 소방관의 기도.ttf", "나눔손글씨 아인맘 손글씨.ttf",
    "나눔손글씨 엄마사랑.ttf", "나눔손글씨 예쁜 민경체.ttf", "나눔손글씨 의미있는 한글.ttf",
    "나눔손글씨 철필글씨.ttf",
    # 2. 온글잎
    "온글잎 공부잘하자나.ttf", "온글잎 김콩해.ttf", "온글잎 바닷바람.ttf",
    "온글잎 위씨리스트.ttf", "온글잎 콘콘체.ttf",
    # 3. MinSans
    "MinSans-Light.ttf", "MinSans-Regular.ttf", "MinSansVF.ttf",
    # 4. Cafe24
    "Cafe24Simplehae-v2.0.ttf", "Cafe24SsurroundAir-v1.1.ttf",
    # 5. Hahmlet
    "Hahmlet-Black.ttf", "Hahmlet-Thin.ttf",
    # 6. 기타
    "MapoHongdaeFreedom.ttf", "UhBee Se_hyun.ttf"
]

# =========================
# 2. 유틸리티 함수
# =========================
def get_clean_folder_name(filename: str) -> str:
    """
    파일명(확장자 제외)에서 특수문자를 제거하여 폴더명으로 변환
    예: "Cafe24Simplehae-v2.0.ttf" -> "Cafe24Simplehaev20"
    """
    stem = Path(filename).stem
    return re.sub(r"[^a-zA-Z0-9가-힣]", "", stem)

# =========================
# 3. 메인 실행
# =========================
if __name__ == "__main__":
    print(f"📋 Processing list of {len(FAILED_FONT_FILENAMES)} failed fonts...")
    print(f"📂 Source: {DATASET_ROOT}")
    print(f"📂 Target: {FAILED_DATASET_ROOT}")
    print("-" * 60)

    moved_count = 0

    for font_file in FAILED_FONT_FILENAMES:
        # 파일명 -> 폴더명 변환
        folder_name = get_clean_folder_name(font_file)

        # 4개의 하위 폴더(back_bw 등) 확인
        for sub in TARGET_SUBFOLDERS:
            src_path = DATASET_ROOT / sub / folder_name
            dest_path = FAILED_DATASET_ROOT / sub / folder_name

            # 원본 데이터셋에 해당 폴더가 존재하면 이동
            if src_path.exists() and src_path.is_dir():
                try:
                    # 목적지 경로 생성
                    dest_path.parent.mkdir(parents=True, exist_ok=True)

                    if dest_path.exists():
                        print(f"⚠️ [Skip] Already exists: {sub}/{folder_name}")
                        # 덮어쓰려면: shutil.rmtree(dest_path) 후 이동
                    else:
                        shutil.move(str(src_path), str(dest_path))
                        print(f"moved 📦: {sub}/{folder_name}")
                        moved_count += 1
                except Exception as e:
                    print(f"❌ Error moving {sub}/{folder_name}: {e}")
            else:
                # 폴더가 없으면 이미 이동했거나 생성이 안 된 것임
                pass

    print("-" * 60)
    print(f"🎉 Done. Moved {moved_count} folders based on the log list.")

📋 Processing list of 24 failed fonts...
📂 Source: C:\Users\nam\Desktop\최종 폰트 데이터셋
📂 Target: C:\Users\nam\Desktop\_Failed_
------------------------------------------------------------
moved 📦: back_bw/나눔손글씨곰신체
moved 📦: back_color_aug/나눔손글씨곰신체
moved 📦: front_bw/나눔손글씨곰신체
moved 📦: front_color_aug/나눔손글씨곰신체
moved 📦: back_bw/나눔손글씨맛있는체
moved 📦: back_color_aug/나눔손글씨맛있는체
moved 📦: front_bw/나눔손글씨맛있는체
moved 📦: front_color_aug/나눔손글씨맛있는체
moved 📦: back_bw/나눔손글씨상해찬미체
moved 📦: back_color_aug/나눔손글씨상해찬미체
moved 📦: front_bw/나눔손글씨상해찬미체
moved 📦: front_color_aug/나눔손글씨상해찬미체
moved 📦: back_bw/나눔손글씨성실체
moved 📦: back_color_aug/나눔손글씨성실체
moved 📦: front_bw/나눔손글씨성실체
moved 📦: front_color_aug/나눔손글씨성실체
moved 📦: back_bw/나눔손글씨소방관의기도
moved 📦: back_color_aug/나눔손글씨소방관의기도
moved 📦: front_bw/나눔손글씨소방관의기도
moved 📦: front_color_aug/나눔손글씨소방관의기도
moved 📦: back_bw/나눔손글씨아인맘손글씨
moved 📦: back_color_aug/나눔손글씨아인맘손글씨
moved 📦: front_bw/나눔손글씨아인맘손글씨
moved 📦: front_color_aug/나눔손글씨아인맘손글씨
moved 📦: back_bw/나눔손글씨엄마사랑
moved 📦: back_color_aug/나눔손글씨엄마사랑


In [1]:
import shutil
import re
from pathlib import Path

# =========================
# ✅ 1. 설정 및 경로
# =========================

# (1) 데이터셋 경로
DATASET_ROOT = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋")
FAILED_DATASET_ROOT = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\_Failed_")
TARGET_SUBFOLDERS = ["back_bw", "back_color_aug", "front_bw", "front_color_aug"]

# (2) 폰트 원본 경로 (추가됨)
FONTS_DIR = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\ref_fonts_ttf")
FAILED_FONTS_DIR = Path(r"C:\Users\nam\Desktop\최종 폰트 데이터셋\_Failed_\ref_fonts_ttf")

# 📝 로그에서 추출한 실패 폰트 목록 (24개)
FAILED_FONT_FILENAMES = [
    # 1. 나눔손글씨
    "나눔손글씨 곰신체.ttf", "나눔손글씨 맛있는체.ttf", "나눔손글씨 상해찬미체.ttf",
    "나눔손글씨 성실체.ttf", "나눔손글씨 소방관의 기도.ttf", "나눔손글씨 아인맘 손글씨.ttf",
    "나눔손글씨 엄마사랑.ttf", "나눔손글씨 예쁜 민경체.ttf", "나눔손글씨 의미있는 한글.ttf",
    "나눔손글씨 철필글씨.ttf",
    # 2. 온글잎
    "온글잎 공부잘하자나.ttf", "온글잎 김콩해.ttf", "온글잎 바닷바람.ttf",
    "온글잎 위씨리스트.ttf", "온글잎 콘콘체.ttf",
    # 3. MinSans
    "MinSans-Light.ttf", "MinSans-Regular.ttf", "MinSansVF.ttf",
    # 4. Cafe24
    "Cafe24Simplehae-v2.0.ttf", "Cafe24SsurroundAir-v1.1.ttf",
    # 5. Hahmlet
    "Hahmlet-Black.ttf", "Hahmlet-Thin.ttf",
    # 6. 기타
    "MapoHongdaeFreedom.ttf", "UhBee Se_hyun.ttf"
]

# =========================
# 2. 유틸리티 함수
# =========================
def get_clean_folder_name(filename: str) -> str:
    """파일명에서 확장자를 떼고 특수문자를 제거하여 폴더명으로 변환"""
    stem = Path(filename).stem
    return re.sub(r"[^a-zA-Z0-9가-힣]", "", stem)

# =========================
# 3. 메인 실행
# =========================
if __name__ == "__main__":
    print(f"📋 Processing list of {len(FAILED_FONT_FILENAMES)} failed fonts...")

    # 폴더 생성
    FAILED_DATASET_ROOT.mkdir(parents=True, exist_ok=True)
    FAILED_FONTS_DIR.mkdir(parents=True, exist_ok=True)

    print("-" * 60)

    moved_font_count = 0
    moved_folder_count = 0

    for font_filename in FAILED_FONT_FILENAMES:
        print(f"\n🔍 Checking: {font_filename}")

        # ---------------------------------------------------------
        # [Step 1] 원본 .ttf 파일 이동
        # ---------------------------------------------------------
        src_font_path = FONTS_DIR / font_filename
        dest_font_path = FAILED_FONTS_DIR / font_filename

        if src_font_path.exists():
            try:
                # 중복 체크
                if dest_font_path.exists():
                    print(f"   ⚠️  [Font] Already exists in failed folder: {font_filename}")
                else:
                    shutil.move(str(src_font_path), str(dest_font_path))
                    print(f"   ✅ [Font] Moved to failed folder.")
                    moved_font_count += 1
            except Exception as e:
                print(f"   ❌ [Font] Error moving file: {e}")
        else:
            print(f"   ℹ️  [Font] File not found in source (already moved?): {font_filename}")

        # ---------------------------------------------------------
        # [Step 2] 데이터셋 폴더 이동 (기존 로직)
        # ---------------------------------------------------------
        folder_name = get_clean_folder_name(font_filename)

        for sub in TARGET_SUBFOLDERS:
            src_dataset_path = DATASET_ROOT / sub / folder_name
            dest_dataset_path = FAILED_DATASET_ROOT / sub / folder_name

            if src_dataset_path.exists() and src_dataset_path.is_dir():
                try:
                    dest_dataset_path.parent.mkdir(parents=True, exist_ok=True)

                    if dest_dataset_path.exists():
                        print(f"   ⚠️  [Data] Folder already exists: {sub}/{folder_name}")
                    else:
                        shutil.move(str(src_dataset_path), str(dest_dataset_path))
                        print(f"   📦 [Data] Moved folder: {sub}/{folder_name}")
                        moved_folder_count += 1
                except Exception as e:
                    print(f"   ❌ [Data] Error moving folder: {e}")

    print("-" * 60)
    print(f"🎉 Summary:")
    print(f"   - Moved Font Files (.ttf): {moved_font_count}")
    print(f"   - Moved Dataset Folders  : {moved_folder_count}")
    print("Done.")

📋 Processing list of 24 failed fonts...
------------------------------------------------------------

🔍 Checking: 나눔손글씨 곰신체.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 맛있는체.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 상해찬미체.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 성실체.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 소방관의 기도.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 아인맘 손글씨.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 엄마사랑.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 예쁜 민경체.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 의미있는 한글.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 나눔손글씨 철필글씨.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 온글잎 공부잘하자나.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 온글잎 김콩해.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 온글잎 바닷바람.ttf
   ✅ [Font] Moved to failed folder.

🔍 Checking: 온글잎 위씨리스트.ttf
   ✅ [Font] Moved to failed folder.

In [3]:
import os
import pandas as pd

target_path = r"C:\Users\nam\Desktop\최종 폰트 데이터셋\ref_fonts_ttf"

# 1. 해당 경로에서 .ttf 파일만 골라내기 (대소문자 무시)
ttf_files = [f for f in os.listdir(target_path)
             if os.path.isfile(os.path.join(target_path, f)) and f.lower().endswith('.ttf')]

# 2. 데이터프레임 생성 및 csv 저장
df = pd.DataFrame(ttf_files, columns=['Font Filename'])

# 저장할 파일명 설정
output_csv = "font_list.csv"
df.to_csv(output_csv, index=False, encoding="utf-8-sig")

print(f"✅ 총 {len(ttf_files)}개의 폰트 파일명을 '{output_csv}'로 저장했습니다.")

✅ 총 126개의 폰트 파일명을 'font_list.csv'로 저장했습니다.
